In [1]:
import lyricwikia
import pandas as pd

df = pd.read_csv("pop_playlist_project.csv")
df.columns = ['Spotify_URI','TrackName','ArtistName','AlbumName','DiscNum','TrackNum','TrackDur','AddedBy','AddedAt']
lyrics = ""

for i in range(len(df)):
    try:
        lyrics_temp = lyricwikia.get_lyrics(df.ArtistName[i], df.TrackName[i])
        lyrics=lyrics+lyrics_temp
    except:
        continue

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

C:\Users\huuki\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
raw_text = lyrics.lower()

# unique mapping of characters to numbers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  744578
Total Vocab:  70


In [6]:
seq_length = 20
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  744558


In [9]:

X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

X = X/float(n_vocab)

y = np_utils.to_categorical(dataY)

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
744558/744558 [==============================] - 1788s 2ms/step - loss: 2.5083
Epoch 2/20
744558/744558 [==============================] - 1887s 3ms/step - loss: 2.0498
Epoch 3/20
744558/744558 [==============================] - 1995s 3ms/step - loss: 1.8740
Epoch 4/20
744558/744558 [==============================] - 1760s 2ms/step - loss: 1.7648
Epoch 5/20
744558/744558 [==============================] - 1632s 2ms/step - loss: 1.6888
Epoch 6/20
744558/744558 [==============================] - 1635s 2ms/step - loss: 1.6315
Epoch 7/20
744558/744558 [==============================] - 1635s 2ms/step - loss: 1.5872
Epoch 8/20
744558/744558 [==============================] - 1636s 2ms/step - loss: 1.5510
Epoch 9/20
744558/744558 [==============================] - 1631s 2ms/step - loss: 1.5234
Epoch 10/20
744558/744558 [==============================] - 1629s 2ms/step - loss: 1.4970
Epoch 11/20
744558/744558 [==============================] - 1650s 2ms/step - loss: 1.4750
Epoch 12

In [13]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
initial_text="these lyrics are for my new great rock song about iupui and these lyrics are for my new great rock song about data"
pattern=[char_to_int[char] for char in initial_text]
pattern=pattern[0:seq_length]

text_gen=""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    prediction = prediction[0]
    index = numpy.random.choice(len(prediction), p=prediction)
    result = int_to_char[index]
    text_gen=text_gen+result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print(text_gen)

 gar love he daco
let me knve me now
i'm sure my noien bnd ereet them bhn't meen 'nh, i'm gonna tale the tay down
i live, grrgver of h love you
to ie ii a glehen
fies ay foy it's to shek oh

art h'l cecriin' out (bh hole mamday
not me tiis temioiss and girlel put love

h meed to know you know
ht's tomeone eloss to me
nh, yeah
you sealldli, you cann babk down
mafe anpne shake on your elessinn is hl toneooe io norh thene
vh'sh sometiing iust a cream i'm aaay
lyst meed ly herd fuasy kust giving up c doiams. loon homesw
when you know sote youare so stnp
shese she l waiting yith you
shoings nucr the world butuin' op these whnl
aut he seidn me cralid
up eor taki
a peteac th you cai
apd would coie on a mistle, sul
may mee see herpe, doyn, your lamehn anl the lovhn's cilccrest so tacrth rkokdn, noh
hot fxehn i bll
nn you're beiggnite me takes bhtw
bc ttedtits, ouel it feel the heart is tp it ayas? this bnm i shene wo go, niver mefe you ceen somou
nume so gipl i'm a geieen
onl thmr c linltn, si

In [15]:
model_json = model.to_json()
with open('model_letter.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('model_letter.h5')
print('Saved model to disk')
    

Saved model to disk


In [16]:
import pickle
pickle_out = open('letterDict.pickle','wb')
pickle.dump(char_to_int,pickle_out)
pickle_out.close()

